In [2]:
from core.utils import Tibanna
from dcicutils import ff_utils
from core.utils import run_workflow
from datetime import datetime
from core.wfr import *

env = 'data'
tibanna = Tibanna(env=env)
ff = ff_utils.fdn_connection(key={"default" : tibanna.ff_keys})
exclude_miseq = True

In [3]:
import time

# for a given experiment set and some parameters like instrument
# print set of files and their partA hic workflow status
# if there are one that are running report the number of running cases
# if there are file pairs that don't have a corresponding part A, report them separately

out_n = "This is an output file of the RepliSeq processing pipeline"
int_n = "This is an intermediate file in the Repliseq processing pipeline"

def step_settings(seq, my_organism):
    genome = ""
    mapper = {'human':'GRCh38','mouse':'GRCm38'}
    genome = mapper.get(my_organism)

    wf_dict =[{
        'wf_name': 'repliseq-parta',
        'wf_uuid': '146da22a-502d-4500-bf57-a7cf0b4b2364',
        "parameters" :  { "nthreads": 4, "memperthread": "2G" },
        'custom_pf_fields': {
            'filtered_sorted_deduped_bam': {
                'genome_assembly': genome,
                'file_type': 'alignment',
                'description': out_n},
            'count_bg': {
                'genome_assembly': genome,
                'file_type': 'read counts',
                'description': out_n}
        }}]
    return wf_dict[seq] 
    
# url for hic exps
exp_types = ['Repli-seq']
#exp_types = ['DAM-ID+seq']

set_url = '/search/?'+'&'.join(['experiments_in_set.experiment_type='+i for i in exp_types])+'&type=ExperimentSetReplicate&limit=all'
run_sets = ff_utils.search_metadata(set_url ,ff_env = env)

add_pc = False
add_rel = False
add_wfr = False

counter = 0
completed = 0
completed_acc = []
all_sets = len(run_sets)
print(str(all_sets)+' total number of sets')

run_sets = [i for i in run_sets if "RepliSeq_Pipeline_0.2.5"  not in i.get('completed_processes', [])]

print(str(all_sets-len(run_sets))+ ' sets completed')


for a_set in run_sets: 
    counter += 1
    print
    fastqpairs, organism, enzyme, bwa_ref, chrsize_ref, enz_ref, f_size = find_pairs(a_set, exclude_miseq, env, tibanna, lookfor = 'single')
    if not bwa_ref or not chrsize_ref:
        print counter, a_set['accession'], organism, enzyme, 'skipping set with not chrsize/bwa index'
        continue  
    print counter, a_set['accession']
    print enzyme, organism
    # part3 = 'done'
    list_release = []
    # set_pairs = []        
    # cycle through the experiments
    for exp in fastqpairs.keys():
        if not fastqpairs.get(exp):
            print(exp, 'does not have any fastq pairs')
            continue
        # Check Part 1 and See if all are okay
        exp_bams = []
        part1 = 'done'
        # part2 = 'done'
        for pair in fastqpairs[exp]:
            #############
            bam = get_wfr_out(pair, 'repliseq-parta 0.2.5', 'bam', env)
            # if run is not successful
            if bam.startswith('no') or not bam:
                part1 = 'not ready'
                if add_wfr:
                    inp_f = {'fastq':pair, 'chromsizes':chrsize_ref, 'bwaIndex':bwa_ref}
                    name_tag = pair.split('/')[2]
                    run_missing_wfr(step_settings(0, organism), inp_f, name_tag, env, tibanna)
            elif bam == 'running':
                part1 = 'still running'
                print('part1 still running')
            # if successful
            else:
                exp_bams.append(bam)
                list_release.append(bam)
        # stop progress to part2 
        if part1 is not 'done':
            print exp, 'has missing Part1 runs'
            part2 = 'not ready'
            part3 = 'not ready'
            continue
        print exp, 'part1 complete'


# 

41 total number of sets
0 sets completed

1 4DNESUZ3Y5GY
None human
4DNEXE23WWO1 part1 complete
4DNEXG6HJJ27 part1 complete

2 4DNESIV168N6
None human
4DNEXVRTS3VG part1 complete
4DNEXELBLZ37 part1 complete

3 4DNESXZ2IL4M
None human
4DNEXKP295JF part1 complete

4 4DNESJOH66ZZ
None human
4DNEXYX58PCM part1 complete

5 4DNESQ3ZDLZM
None human
4DNEXCTB91SG part1 complete

6 4DNES98TS7X2
None human
4DNEXXIXTTXI part1 complete

7 4DNESX6MGDN2
None human
4DNEXLVWLA33 part1 complete

8 4DNES6O1SJ4O
None human
4DNEX1BWBWFA part1 complete

9 4DNESWS6UT74
None human
4DNEXZPBA82F part1 complete

10 4DNESBAF7EW5
None human
4DNEXDWEK1ME part1 complete

11 4DNES9OZNM6T
None human
4DNEXTM3PBS3 part1 complete

12 4DNESDFQDLQJ
None human
4DNEX8K5TT7Y part1 complete

13 4DNES3DSRT51
None human
4DNEXS77PTUK part1 complete

14 4DNES7OJ7U7X
None human
4DNEXQ4EXW3K part1 complete

15 4DNESLS88OS7
None human
4DNEXEQB7RNA part1 complete

16 4DNESDUJHQ6J
None human
4DNEX5M9XSGK part1 complete

17 4DNESCBR4664

In [ ]:
        #check if part 2 is run already, it not start the run
#         exp_com_bam = []
#         exp_pairs = []
#         for bam in exp_bams:
#             com_bam = get_wfr_out(bam, 'hi-c-processing-bam 0.2.5', 'bam', ff)
#             pairs = get_wfr_out(bam, 'hi-c-processing-bam 0.2.5', 'pairs', ff)
#             # try to run if missing
#             if pairs.startswith('no') or not pairs:
#                 part2 = 'not ready'
#                 part3 = 'not ready'
                
#             elif pairs == 'running':
#                 print(bam)
#                 part2 = 'still running'
#                 part3 = 'not ready'
                
#             else:
#                 exp_com_bam.append(com_bam)
#                 exp_pairs.append(pairs)
                
#         # if still running, skip to next experiment
#         if part2 == 'still running':
#             print('part2 still running')
#             continue
        
#         # make sure all bams went through the same wfr and produces same file
#         if part2 != 'done' or len(list(set(exp_com_bam))) != 1 or len(list(set(exp_pairs))) !=1:
#             print exp, 'Part2 did not complete'
#             part3 = 'not ready' 
        
#             if add_wfr:
#                 if not chrsize_ref:
#                     print 'not yet usable', organism
#                     continue
#                 # make sure no duplicates
#                 inp_f = {'input_bams':exp_bams, 'chromsize':chrsize_ref}           
#                 run_missing_wfr(wf_dict[1], inp_f, exp, ff, env, tibanna)   
#             continue
            
#         # add bam and pairs to exp proc file
#         list_release.extend([exp_com_bam[0],exp_pairs[0]])
#         if add_pc:
#             add_processed_files(exp, [exp_com_bam[0],exp_pairs[0]], ff)
        
#         print exp, 'part2 complete'
#         set_pairs.append(exp_pairs[0])
    
#     if part3 != 'done':
#         print 'Part3 not ready'
#         continue
    
#     if not set_pairs:
#         print 'no pairs can be produced from this set'
#         continue
        
#     merged_pairs = []
#     for set_pair in set_pairs:
#         merged_pair = get_wfr_out(set_pair, 'hi-c-processing-pairs 0.2.5', 'pairs', ff)
#         hic = get_wfr_out(set_pair, 'hi-c-processing-pairs 0.2.5', 'hic', ff)
#         mcool = get_wfr_out(set_pair, 'hi-c-processing-pairs 0.2.5', 'mcool', ff)
#         normvec = get_wfr_out(set_pair, 'hi-c-processing-pairs 0.2.5', 'normvector_juicerformat', ff)
#         if merged_pair.startswith('no') or not merged_pair:
#             part3 = 'not ready'
#             break
#         elif merged_pair == 'running':
#             part3 = 'still running'
#             break
#         else:
#             merged_pairs.append(merged_pair)
    
    
#     # if part3 is still running report it, and skip the rest of the script
#     if part3 == 'still running':
#         print 'part3', part3
#         continue        
                
#     if part3 != 'done' or len(list(set(merged_pairs))) != 1:
#         print a_set['accession'], 'is missing Part3'
        
#         # if it is not run, and add_wfr is true, go for it, then skip the rest of the script
#         if add_wfr:
#             if not chrsize_ref:
#                 print 'not yet usable', organism
#                 continue

#             if not enz_ref:
#                 print 'restriction enzyme not ready for', organism, enzyme
#                 continue
#             inp_f = {'input_pairs':set_pairs, 'chromsizes':chrsize_ref, 'restriction_file': enz_ref} 
#             run_missing_wfr(wf_dict[2], inp_f, a_set['accession'], ff, env, tibanna)
#         continue
#     #####
#     #add competed flag to experiment
#     if add_pc and add_rel:
#         ff_utils.patch_metadata({"completed_processes":["RepliSeq_Pipeline_0.2.5"]}, obj_id=a_set['accession'] ,connection=ff)
    
#     # add processed files to set
#     list_release.extend([merged_pair, hic, mcool, normvec])
#     if add_pc:
#         add_processed_files(a_set['accession'], [merged_pair, hic, mcool, normvec], ff)
    
#     #release files and wfrs
#     if add_rel:
#         release_files(a_set['accession'], list(set(list_release)), ff)
    
#     completed += 1
#     completed_acc.append(a_set['accession'])
#     print a_set['accession'], 'part3 complete'

    
# print completed
# print completed_acc